In [ ]:
!pip3 install jupyter-tabnine --user
!pip install jupyter-contrib-nbextensions

!jupyter nbextension install --py jupyter_tabnine --user
!jupyter nbextension enable --py jupyter_tabnine --user
!jupyter serverextension enable --py jupyter_tabnine --user

In [ ]:
!pip install selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time 





In [ ]:
# Specify the path to the ChromeDriver
service = Service("C:/Users/welcome/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service)

# Open the LinkedIn website
driver.get("https://www.linkedin.com/login")


In [ ]:
# Wait for the page to load
time.sleep(2)

# Locate the username and password fields using the updated syntax
username = driver.find_element(By.XPATH, "//input[@name='session_key']")
password = driver.find_element(By.XPATH, "//input[@name='session_password']")

# Enter your LinkedIn credentials
username.send_keys("Username")#enter your username
password.send_keys("Password")#enter you password

# Wait a moment before submitting
time.sleep(2)

# Locate the submit button and click it
submit = driver.find_element(By.XPATH, "//button[@type='submit']")
driver.execute_script("arguments[0].click();",submit )
time.sleep(2)

In [ ]:

# Function to send a message
def send_message():
    # Go to LinkedIn search page for connections
    driver.get("https://www.linkedin.com/search/results/people/?network=%5B%22F%22%5D&origin=FACETED_SEARCH")
    time.sleep(2)  # Give time for the page to load fully

    # Function to safely get message buttons
    def get_message_buttons():
        all_buttons = driver.find_elements(By.TAG_NAME, "button")
        return [btn for btn in all_buttons if btn.text == "Message"]

    # Main loop to process each message
    for i in range(1, len(get_message_buttons()) - 2):  # Adjust this range based on your needs
        try:
            message_buttons = get_message_buttons()
            # Click the message button using JavaScript to avoid interaction issues
            driver.execute_script("arguments[0].click();", message_buttons[i])
            time.sleep(2)

            # Wait for the message box to be clickable and interact with it
            try:
                main_div = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[starts-with(@class,'msg-form__msg-content-container')]"))
                )
                driver.execute_script("arguments[0].click();", main_div)

                # Check if there's any predefined text
                paragraphs = driver.find_elements(By.TAG_NAME, "p")
                if paragraphs:  # If any text exists in the message box
                    pre_existing_text = paragraphs[0].text.strip()
                    if pre_existing_text:
                        driver.execute_script("arguments[0].innerText = '';", paragraphs[0])
                        print("Predefined message detected and cleared.")

                # Find the message input area and send the message
                message_box = driver.find_element(By.XPATH, "//div[starts-with(@class,'msg-form__contenteditable')]")
                message_box.clear()
                time.sleep(1)

                # Define the message content
                message_lines = [
                    "Hello,",
                    "",
                    "I hope this message finds you well! I wanted to reach out and see how you've been doing since we last connected.",
                    "",
                    "I'm excited to see how things are progressing on your end and would love to catch up when you have time.",
                    "",
                    "Best regards,",
                    "Vikramaditya Singh"
                ]

                # Send the message line by line
                for line in message_lines:
                    message_box.send_keys(line)
                    message_box.send_keys("\n")
                    time.sleep(0.5)

                time.sleep(2)

                # Find and click the submit button
                submit = driver.find_element(By.XPATH, "//button[@type='submit']")
                driver.execute_script("arguments[0].click();", submit)
                print(f"Message sent successfully to connection {i}.")

                # Close the message window
                close_button = driver.find_element(By.XPATH, "//button[contains(@class, 'msg-overlay-bubble-header__control') and contains(@class, 'artdeco-button--circle')]")
                ActionChains(driver).move_to_element(close_button).click().perform()
                print("Close button clicked successfully.")
                time.sleep(2)

            except Exception as e:
                print(f"Error occurred for connection {i}: {e}")

        except StaleElementReferenceException:
            print("Encountered StaleElementReferenceException. Retrying...")
            message_buttons = get_message_buttons()  # Relocate the elements and retry

        # After sending each message, refresh the page and reload LinkedIn connections
        driver.get("https://www.linkedin.com/search/results/people/?network=%5B%22F%22%5D&origin=FACETED_SEARCH")
        time.sleep(5)  # Allow time for the page to reload

# Main execution
if __name__ == "__main__":
    try:
        send_message()
    finally:
        driver.quit()
